# Import

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

# Data

In [ ]:
# Load data
path1 = "/content/drive/MyDrive/Thesis/Data/Data/Cleaned_Price.csv"
price = pd.read_csv(path1, sep=',')

path2 = "/content/drive/MyDrive/Thesis/Data/Data/FRED MD - National level explanatory data.csv"
variables = pd.read_csv(path2, sep=';')

# Preprocessing data

In [ ]:
# Preprossing data
price['Date'] = pd.to_datetime(price['Date'])
variables['Date'] = pd.to_datetime(variables['Date'])
variables.set_index('Date', inplace=True)

In [ ]:
datetime_index_col = 'Date'

variables['HWI'] = variables['HWI'].astype(float)

for x in variables.columns:
    if variables[x].dtype == 'object':
        variables[x] = variables[x].str.replace(',' , '.')
        variables[x] = pd.to_numeric(variables[x], errors='coerce')

In [ ]:
# Setting the start date of data
specific_date = '1975-01-01'
specific_date = pd.Timestamp(specific_date)
variables = variables.loc[specific_date:]

In [ ]:
# Counting missing values from FRED-MD
non_null_counts = variables.count()
columns_with_missing_values = non_null_counts[non_null_counts != 588]

print("Columns with missing values:")
for column, count in columns_with_missing_values.items():
    print(f"{column}: {588 - count} missing values")

print("Total number of columns with missing values:", len(columns_with_missing_values))

In [ ]:
# Dropping values that is recorded later than the dataset start
drop = ['ACOGNO', 'UMCSENTx']
variables.drop(columns=drop, inplace=True)

In [ ]:
# Checking if the dropped variables are actually dropped
non_null_counts = variables.count()
columns_with_missing_values = non_null_counts[non_null_counts != 588]

print("Columns with missing values:")
for column, count in columns_with_missing_values.items():
    print(f"{column}: {588 - count} missing values")

print("Total number of columns with missing values:", len(columns_with_missing_values))

In [ ]:
# Forward imputating in variables that has max 6 missing values
variables = variables.ffill(axis=0)
selected_columns = variables[['CP3Mx', 'COMPAPFFx', 'S&P div yield', 'S&P PE ratio']]
selected_columns

In [ ]:
# Checking the forward imputation mechanism

non_null_counts = variables.count()
columns_with_missing_values = non_null_counts[non_null_counts != 588]

print("Columns with missing values:")
for column, count in columns_with_missing_values.items():
    print(f"{column}: {588 - count} missing values")

print("Total number of columns with missing values:", len(columns_with_missing_values))

In [ ]:
# Concat FRED-MD data on each state
repeat = pd.concat([variables] * 51, ignore_index=True)
result = pd.concat([price, repeat], axis=1)

# Regions Data from FRED-MD

In [ ]:
# Load data
path3 = "/content/drive/MyDrive/Thesis/Data/Data/State Regions.csv"
regions = pd.read_csv(path3, sep=';')

In [ ]:
# Merge the region configuration on each state
merged_df = pd.merge(result, regions[['State', 'Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)']], left_on='GEO_Name', right_on='State', how='left')
merged_df.drop('State', axis=1, inplace=True)

## Making HOUST into 1 column based on their region

In [ ]:
# Placeholder for 'HOUST'
merged_df['houst'] = None

In [ ]:
HOUST_column = merged_df.pop('HOUST')
merged_df['HOUST'] = HOUST_column
HOUSTNE_column = merged_df.pop('HOUSTNE')
merged_df['HOUSTNE'] = HOUSTNE_column
HOUSTMW_column = merged_df.pop('HOUSTMW')
merged_df['HOUSTMW'] = HOUSTMW_column
HOUSTS_column = merged_df.pop('HOUSTS')
merged_df['HOUSTS'] = HOUSTS_column
HOUSTW_column = merged_df.pop('HOUSTW')
merged_df['HOUSTW'] = HOUSTW_column

In [ ]:
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 1, 'houst'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 1, 'HOUSTNE']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 2, 'houst'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 2, 'HOUSTMW']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 3, 'houst'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 3, 'HOUSTS']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 4, 'houst'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 4, 'HOUSTW']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 5, 'houst'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 5, 'HOUST']

In [ ]:
# Dropping excess columns
columns_to_drop = ['HOUST', 'HOUSTNE', 'HOUSTMW', 'HOUSTS', 'HOUSTW']
merged_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
# Renaming the variable
merged_df.rename(columns={'houst': 'HOUST'}, inplace=True)

## Making PERMIT into 1 column based on their region

In [ ]:
# Placeholder for 'PERMIT'
merged_df['permit'] = None

In [ ]:
PERMIT_column = merged_df.pop('PERMIT')
merged_df['PERMIT'] = PERMIT_column
PERMITNE_column = merged_df.pop('PERMITNE')
merged_df['PERMITNE'] = PERMITNE_column
PERMITMW_column = merged_df.pop('PERMITMW')
merged_df['PERMITMW'] = PERMITMW_column
PERMITS_column = merged_df.pop('PERMITS')
merged_df['PERMITS'] = PERMITS_column
PERMITW_column = merged_df.pop('PERMITW')
merged_df['PERMITW'] = PERMITW_column

In [ ]:
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 1, 'permit'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 1, 'PERMITNE']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 2, 'permit'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 2, 'PERMITMW']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 3, 'permit'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 3, 'PERMITS']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 4, 'permit'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 4, 'PERMITW']
merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 5, 'permit'] = merged_df.loc[merged_df['Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'] == 5, 'PERMIT']

In [ ]:
# Dropping excess variables
columns_to_drop = ['PERMIT', 'PERMITNE', 'PERMITMW', 'PERMITS', 'PERMITW']
merged_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
# Renaming the variable
merged_df.rename(columns={'permit': 'PERMIT'}, inplace=True)

# Load and transformation of Sentiment variables

In [ ]:
# Load data
path6 = "/content/drive/MyDrive/Thesis/Data/Data/Michigan Survey - Sentiment - All.csv"
sentiment = pd.read_csv(path6, sep=';')

sentiment['Date'] = pd.to_datetime(sentiment['Date'])

In [ ]:
# Merge dataframes from FRED-MD and Michigan Survey
merged_data = pd.merge(merged_df, sentiment, on=['Date', 'Region (1=NE, 2=MW, 3=S, 4=W, 5=USA)'], how='left')
merged_data.dropna(inplace=True)
ALL_DATA = merged_data.reset_index(drop=True)

In [ ]:
# Dropping 'GEO_Types' and 'Index_NSA'
columns = ['GEO_Type', 'Index_NSA']
ALL_DATA.drop(columns=columns, inplace=True)
final = ALL_DATA.reset_index(drop=True)

# Taking Ln of Index_SA

In [ ]:
final['Ln(Index_SA)'] = np.log(final['Index_SA'])

# Calculating Log Returns at different horizons


In [ ]:
# Defining all unique states
state_abbreviations = final['GEO_Name'].unique()

horizons = [1, 3, 6, 12]

def calculate_log_returns(df, horizons):
    for h in horizons:
        df[f'Log_Return_h{h}'] = df['Ln(Index_SA)'] - df['Ln(Index_SA)'].shift(h)
    return df

# Applying the function to each state separately and concat the results
final_with_returns = pd.concat(
    [calculate_log_returns(final[final['GEO_Name'] == state_abbr].copy(), horizons) for state_abbr in state_abbreviations]
)

final_with_returns.reset_index(drop=True, inplace=True)
final_with_returns.set_index('Date', inplace=True)

# Saving the merged data frame as CSV

In [ ]:
final_with_returns.to_csv("/content/drive/MyDrive/Thesis/Data/Merging CleanPrice & Features.csv")

In [ ]:
# END